In [1]:
import os 
import datetime
from pathlib import Path
from dotenv import load_dotenv, find_dotenv

basepath = Path(os.getcwd())
# make sure your working directory is the repository root.
if basepath.name != "idp-radio-1":
    os.chdir(basepath.parent.parent)
load_dotenv(find_dotenv())

%load_ext autoreload
%autoreload 2
os.getcwd()

'/srv/idp-radio-1'

In [2]:
import os 
import math
import tensorflow as tf
from pathlib import Path
from src.metrics.losses import WeightedBinaryCrossentropy, SimpleWeightedBinaryCrossentropy
from src.metrics.metrics import F2Score, SingleClassMetric
import keras.backend.tensorflow_backend as K
from src.architectures.benchmarks.benchmark import Benchmark, Experiment
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from src.architectures.simple.simple_base import SimpleBaseArchitecture


Using TensorFlow backend.


In [3]:
# Specify which GPU(s) to use
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Or 2, 3, etc. other than 0

config = tf.compat.v1.ConfigProto(allow_soft_placement=True, log_device_placement=True)
config.gpu_options.allow_growth = True
tf.compat.v1.Session(config=config)


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: GeForce RTX 2080 Ti, pci bus id: 0000:42:00.0, compute capability: 7.5



In [4]:
y_pred = tf.constant([[0.,1.,0.], [0.,0.25,1.], [1.,0.,0.75]])
y_true = tf.constant([[0,1.0,1.0], [1.0,0,1.0], [1.,1.,0]])
weights = tf.constant([2, 0.5, 10.])
print(y_pred)
print(y_true)
print(weights)

tf.Tensor(
[[0.   1.   0.  ]
 [0.   0.25 1.  ]
 [1.   0.   0.75]], shape=(3, 3), dtype=float32)
tf.Tensor(
[[0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]], shape=(3, 3), dtype=float32)
tf.Tensor([ 2.   0.5 10. ], shape=(3,), dtype=float32)


In [5]:
# sanity checks
# 1. throw all classes at the same time in backend cross_entropy
print("All classes in backend cross_entropy")
print(K.binary_crossentropy(y_true, y_pred))
print(K.mean(K.binary_crossentropy(y_true, y_pred)))
print()

# 2. throw all classes at the same time in keras loss cross_entropy
print("All classes in keras loss cross_entropy")
print(tf.keras.losses.BinaryCrossentropy()(y_true, y_pred))
print()

# 3. apply backend cross entropy for each class seperately:
print("backend cross entropy for each class seperately:")
print(K.mean(K.binary_crossentropy(y_true, y_pred), axis=1))
print()

for i in range(len(y_pred)):
    #print("y_pred", y_pred[:, i])
    #print("y_true", y_true[:, i])
    print("cross_entropy for class ", i)
    print(tf.keras.losses.BinaryCrossentropy()(y_true[:, i], y_pred[:, i]))
    print()
    
# 4. check simple custom weighted cross entropy with weights set to 1
print("simple custom weighted cross entrop")
simple_weighted_bce_fn = SimpleWeightedBinaryCrossentropy(tf.constant([1., 1., 1.]))
print(simple_weighted_bce_fn(y_true, y_pred))
print()

# 5. check custom weighted cross entropy with pos and negative weights set to 1
print("custom weighted cross entrop")
weighted_bce_fn = WeightedBinaryCrossentropy(tf.constant([1., 1., 1.]),  tf.constant([1., 1., 1.]))
print(weighted_bce_fn(y_true, y_pred))
print()


All classes in backend cross_entropy
tf.Tensor(
[[-0.        -0.        15.424949 ]
 [15.424949   0.2876819 -0.       ]
 [-0.        15.424949   1.386294 ]], shape=(3, 3), dtype=float32)
tf.Tensor(5.3276467, shape=(), dtype=float32)

All classes in keras loss cross_entropy
tf.Tensor(5.3276467, shape=(), dtype=float32)

backend cross entropy for each class seperately:
tf.Tensor([5.1416497 5.2375436 5.603748 ], shape=(3,), dtype=float32)

cross_entropy for class  0
tf.Tensor(5.1416497, shape=(), dtype=float32)

cross_entropy for class  1
tf.Tensor(5.2375436, shape=(), dtype=float32)

cross_entropy for class  2
tf.Tensor(5.603748, shape=(), dtype=float32)

simple custom weighted cross entrop
tf.Tensor(5.3276467, shape=(), dtype=float32)

custom weighted cross entrop
tf.Tensor(5.3276467, shape=(), dtype=float32)



In [6]:
card_acc = SingleClassMetric(tf.keras.metrics.BinaryAccuracy(name="accuracy"), 2, class_name="cardiomegaly")
dev_acc = SingleClassMetric(tf.keras.metrics.BinaryAccuracy(name="accuracy"), 0, class_name="device")


In [7]:

class_num = y_pred.shape[-1]
sample_num = y_pred.shape[-2]
print(class_num)
print(sample_num)
sample_weight = tf.repeat(tf.expand_dims(tf.one_hot(1, class_num),axis=0), sample_num, axis=0)
#sample_weight = tf.expand_dims(tf.one_hot(2, class_num),axis=0)
sample_weight

3
3


<tf.Tensor: shape=(3, 3), dtype=float32, numpy=
array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.]], dtype=float32)>

In [11]:

card_acc.update_state(y_true, y_pred)
card_acc.result()

sample_weight tf.Tensor([0. 0. 1.], shape=(3,), dtype=float32)


0.33333334

In [15]:

dev_acc.update_state(y_true, y_pred)
dev_acc.result()

<tf.Tensor: shape=(), dtype=float32, numpy=0.6666667>

In [16]:
metrics = [tf.keras.metrics.AUC(multi_label=True, name="auc"),
           tf.keras.metrics.Precision(name="precision"),
           tf.keras.metrics.Recall(name="recall"),
           F2Score(name="f2_score"),
           tf.keras.metrics.BinaryAccuracy(name="binary_accuracy")
          ]

chexpert_columns = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly', 'Lung Opacity', 'Lung Lesion',
                       'Edema', 'Consolidation', 'Pneumonia', 'Atelectasis', 'Pneumothorax',
                       'Pleural Effusion', 'Pleural Other', 'Fracture', 'Support Devices']

chexpert_benchmark = Benchmark(Path(os.environ.get("CHEXPERT_DEV_DATASET_DIRECTORY")),
                                  chexpert_columns,
                                  "Chexpert dev small batches",
                                  epochs=10,
                                  train_labels="train.csv",
                                  path_column="Path",
                                  split_group='patient_id',
                                  batch_size=16,
                                  metrics = metrics,
                                  loss = WeightedBinaryCrossentropy(tf.constant([1.,]*len(chexpert_columns)))
                                  #loss = weighted_binary_cross_entropy(tf.constant([1., 1., 1.]))
                              )
chexpert_benchmark.summary()

TypeError: __init__() missing 1 required positional argument: 'negative_class_weights'

In [ ]:
import numpy as np
np.unique()[1:]

In [ ]:
sparse = chexpert_benchmark.traingen.get_labels() * (np.array(range(14)) + 1)

In [ ]:
compute_class_weight(chexpert_benchmark.traingen)

In [ ]:
compute_class_weight(chexpert_benchmark.traingen)

In [ ]:
sparse_negativ_labels = (1 - chexpert_benchmark.traingen.get_labels()) * (np.array(range(14)) + 1)

In [ ]:
 (chexpert_benchmark.traingen.get_labels().shape[0]) /  len(sparse_negativ_labels[sparse_negativ_labels == 1])

In [ ]:
np.array([np.float("nan"),2,3]) * np.array(range(3)) + 1